In [4]:
import json
from google import genai


In [2]:
client = genai.Client()

In [11]:
ddl = """
   CREATE TABLE `country` (
  `Code` varchar(50) DEFAULT NULL,
  `Name` varchar(50) DEFAULT NULL,
  `Continent` varchar(50) DEFAULT NULL,
  `Region` varchar(50) DEFAULT NULL,
  `SurfaceArea` int DEFAULT NULL,
  `IndepYear` int DEFAULT NULL,
  `Population` int DEFAULT NULL,
  `LifeExpectancy` double DEFAULT NULL,
  `GNP` double DEFAULT NULL,
  `GNPOld` int DEFAULT NULL,
  `LocalName` varchar(50) DEFAULT NULL,
  `GovernmentForm` varchar(50) DEFAULT NULL,
  `HeadOfState` varchar(50) DEFAULT NULL,
  `Capital` int DEFAULT NULL,
  `Code2` varchar(50) DEFAULT NULL)
"""

instructions = """ 
Please convert the below ddls into the json formated table schema, with all the constraints, primary keys, foreign keys, etc., do not miss any type of constraints.
make sure that you only return me the json object with all the detail in it , I dont need any extra explination just pure json object is required
"""

payload = instructions + ddl

In [14]:
pre_context = client.models.generate_content(model='gemini-2.5-pro',contents =payload)

In [ ]:
pre_context = pre_context.text

In [30]:
final = f"""
for the below json object ddl, return me a json object with 
tests : list of tests, each with name, description, and SQL or rule, this is for the post load checks.
Data Quality : this is for the pre load checksdata, need data contract definitions, all the cheks that needs to be done for the staging data before getting loaded to the final table such that 
the load doesnt fail or no incorrect data is loaded.
docs_markdown : a confluence style markdown document with the explanation of what kind of table is it and followed by all the important details of the tables, if there are multiple tables answer then individually one by one.

the json object should look like this:
{{
    "tests": [
        {{
            "name": "test_name",
            "description": "test_description",
            "sql": "test_sql"
        }}
    ],
    "data_quality": {{
        "data_contract": "data_contract",
        "checks": [
            {{
                "name": "check_name",
                "description": "check_description",
                "sql": "check_sql"
            }}
        ]
    }},
    "docs_markdown": "docs_markdown"
}}

you need to use the below json object to generate the json object:
{pre_context}
"""


In [33]:
dq_checks = client.models.generate_content(model='gemini-2.0-flash',contents=final)

In [36]:
print(dq_checks.text)

```json
{
  "tests": [
    {
      "name": "test_country_row_count",
      "description": "Verify that the country table has a reasonable number of rows after the load.",
      "sql": "SELECT COUNT(*) FROM country WHERE code IS NOT NULL;"
    },
    {
      "name": "test_country_continent_values",
      "description": "Check for valid values in the Continent column.",
      "sql": "SELECT COUNT(*) FROM country WHERE Continent NOT IN ('Asia', 'Europe', 'North America', 'Africa', 'Oceania', 'Antarctica', 'South America');"
    },
    {
      "name": "test_country_code_uniqueness",
      "description": "Ensure that country codes are unique",
      "sql": "SELECT COUNT(*) FROM (SELECT Code FROM country GROUP BY Code HAVING COUNT(*) > 1) AS DuplicateCodes;"
    }
  ],
  "data_quality": {
    "data_contract": "The country table should contain accurate and complete information about countries worldwide, including their codes, names, geographic details, and demographic statistics.",
    "check